In [2]:
# %%
# 피처 엔지니어링 - Naive Bayes Customer Analysis (수정 버전)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

print("피처 엔지니어링 시작! (원본 데이터 + 할인 레코드 활용)")

# %%
# 1. 원본 데이터 로딩 (할인 레코드 포함)
raw_path = '../data/raw/'

# 원본 transaction_data 사용 (QUANTITY=0 포함)
transaction_data = pd.read_csv(f'{raw_path}transaction_data.csv')
coupon = pd.read_csv(f'{raw_path}coupon.csv')
hh_demographic = pd.read_csv(f'{raw_path}hh_demographic.csv')
product = pd.read_csv(f'{raw_path}product.csv')
campaign_table = pd.read_csv(f'{raw_path}campaign_table.csv')

print(f"transaction_data: {len(transaction_data):,}건 (할인 레코드 포함)")
print(f"고유 가구 수: {transaction_data['household_key'].nunique()}")
print(f"분석 기간: {transaction_data['DAY'].min()} ~ {transaction_data['DAY'].max()}일")

# 쿠폰 중복만 제거
coupon_clean = coupon.drop_duplicates()
print(f"coupon 중복 제거: {len(coupon)} → {len(coupon_clean)}")

# %%
# 2. 거래 데이터 분리 및 기본 정제

print("=== 거래 데이터 분리 ===")

# 2-1. 정상 구매 거래 (QUANTITY > 0)
normal_transactions = transaction_data[transaction_data['QUANTITY'] > 0].copy()
print(f"정상 구매 거래: {len(normal_transactions):,}건")

# 2-2. 할인/프로모션 레코드 (QUANTITY = 0)
discount_records = transaction_data[transaction_data['QUANTITY'] == 0].copy()
print(f"할인/프로모션 레코드: {len(discount_records):,}건")

# 2-3. 극단적 아웃라이어만 제거 (정상 거래에서)
sales_q995 = normal_transactions['SALES_VALUE'].quantile(0.995)
quantity_q995 = normal_transactions['QUANTITY'].quantile(0.995)

print(f"\n아웃라이어 제거 기준:")
print(f"  거래금액 > ${sales_q995:.2f}")
print(f"  수량 > {quantity_q995:.0f}")

# 극단 아웃라이어 제거
normal_clean = normal_transactions[
    (normal_transactions['SALES_VALUE'] <= sales_q995) & 
    (normal_transactions['QUANTITY'] <= quantity_q995)
].copy()

print(f"정상 거래 정제: {len(normal_transactions):,} → {len(normal_clean):,}건")

# %%
# 3. 정상 구매 기반 RFM 특성

print("=== 정상 구매 기반 RFM 특성 생성 ===")

# 가구별 기본 통계 (정상 거래만)
customer_features = normal_clean.groupby('household_key').agg({
    'DAY': ['min', 'max', 'count'],
    'SALES_VALUE': ['sum', 'mean', 'std', 'count'],
    'QUANTITY': ['sum', 'mean'],
    'BASKET_ID': 'nunique'
}).round(2)

customer_features.columns = [
    'first_purchase_day', 'last_purchase_day', 'purchase_days',
    'total_sales', 'avg_sales', 'std_sales', 'total_transactions',
    'total_quantity', 'avg_quantity', 'total_visits'
]

# RFM 계산
analysis_end_day = transaction_data['DAY'].max()
customer_features['recency'] = analysis_end_day - customer_features['last_purchase_day']
customer_features['frequency'] = customer_features['total_visits']
customer_features['monetary'] = customer_features['total_sales']

# 추가 파생 변수
customer_features['customer_lifetime_days'] = customer_features['last_purchase_day'] - customer_features['first_purchase_day'] + 1
customer_features['avg_basket_value'] = customer_features['total_sales'] / customer_features['total_visits']
customer_features['avg_items_per_visit'] = customer_features['total_quantity'] / customer_features['total_visits']

print(f"RFM 기반 고객 특성: {len(customer_features)}명")

# %%
# 4. 할인/프로모션 반응 특성 (새로 추가!)

print("=== 할인/프로모션 반응 특성 생성 ===")

# 4-1. 할인 레코드 기반 특성
if len(discount_records) > 0:
    discount_features = discount_records.groupby('household_key').agg({
        'COUPON_DISC': ['count', 'sum', 'mean'],
        'RETAIL_DISC': ['count', 'sum', 'mean'],
        'DAY': 'nunique'  # 할인 사용한 날 수
    }).round(2)
    
    discount_features.columns = [
        'coupon_usage_count', 'total_coupon_discount', 'avg_coupon_discount',
        'retail_disc_count', 'total_retail_discount', 'avg_retail_discount',
        'discount_days'
    ]
    
    # 할인 민감도 지표
    discount_features['total_discount_amount'] = abs(discount_features['total_coupon_discount']) + abs(discount_features['total_retail_discount'])
    discount_features['discount_frequency'] = discount_features['coupon_usage_count'] + discount_features['retail_disc_count']
    
    print(f"할인 사용 고객: {len(discount_features)}명")
else:
    discount_features = pd.DataFrame()

# 4-2. 정상 거래에서의 할인 사용 패턴
normal_discount_usage = normal_clean[
    (normal_clean['RETAIL_DISC'] != 0) | (normal_clean['COUPON_DISC'] != 0)
].groupby('household_key').agg({
    'RETAIL_DISC': ['count', 'sum'],
    'COUPON_DISC': ['count', 'sum']
}).round(2)

if len(normal_discount_usage) > 0:
    normal_discount_usage.columns = [
        'normal_retail_disc_count', 'normal_retail_disc_total',
        'normal_coupon_disc_count', 'normal_coupon_disc_total'
    ]
    print(f"정상 거래 중 할인 사용 고객: {len(normal_discount_usage)}명")

# 4-3. 무료 상품 이용 (SALES_VALUE = 0, QUANTITY > 0)
free_products = normal_clean[normal_clean['SALES_VALUE'] == 0]
if len(free_products) > 0:
    free_product_usage = free_products.groupby('household_key').agg({
        'QUANTITY': ['count', 'sum'],
        'DAY': 'nunique'
    }).round(2)
    
    free_product_usage.columns = ['free_product_transactions', 'free_product_quantity', 'free_product_days']
    print(f"무료 상품 이용 고객: {len(free_product_usage)}명")
else:
    free_product_usage = pd.DataFrame()

# %%
# 5. 타겟 변수 생성

print("=== 타겟 변수 생성 ===")

# 고가치 고객 기준 (정상 거래 기준)
monetary_80th = customer_features['monetary'].quantile(0.8)
frequency_80th = customer_features['frequency'].quantile(0.8)

print(f"상위 20% 기준:")
print(f"  총 구매액: ${monetary_80th:.2f}")
print(f"  방문 횟수: {frequency_80th:.1f}회")

# 다양한 타겟 변수 생성
customer_features['high_value_monetary'] = (customer_features['monetary'] >= monetary_80th).astype(int)
customer_features['high_value_frequency'] = (customer_features['frequency'] >= frequency_80th).astype(int)
customer_features['high_value_combined'] = (
    (customer_features['monetary'] >= monetary_80th) | 
    (customer_features['frequency'] >= frequency_80th)
).astype(int)

# RFM 스코어 기준
def create_rfm_score(df):
    df['R_score'] = pd.qcut(df['recency'], 5, labels=[5,4,3,2,1], duplicates='drop')
    df['F_score'] = pd.qcut(df['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5], duplicates='drop')
    df['M_score'] = pd.qcut(df['monetary'].rank(method='first'), 5, labels=[1,2,3,4,5], duplicates='drop')
    df['RFM_score'] = df['R_score'].astype(int) + df['F_score'].astype(int) + df['M_score'].astype(int)
    return df

customer_features = create_rfm_score(customer_features)
customer_features['high_value_rfm'] = (customer_features['RFM_score'] >= 12).astype(int)

print(f"\n타겟 변수 분포:")
print(f"고가치 (구매액): {customer_features['high_value_monetary'].mean()*100:.1f}%")
print(f"고가치 (방문횟수): {customer_features['high_value_frequency'].mean()*100:.1f}%")
print(f"고가치 (복합): {customer_features['high_value_combined'].mean()*100:.1f}%")
print(f"고가치 (RFM): {customer_features['high_value_rfm'].mean()*100:.1f}%")

# %%
# 6. 카테고리 선호도 특성

print("=== 카테고리 선호도 특성 생성 ===")

# 정상 거래에 상품 정보 결합
normal_with_product = normal_clean.merge(product[['PRODUCT_ID', 'DEPARTMENT']], on='PRODUCT_ID', how='left')

# 가구별 카테고리 구매 패턴
category_spending = normal_with_product.groupby(['household_key', 'DEPARTMENT'])['SALES_VALUE'].sum().reset_index()
category_pivot = category_spending.pivot(index='household_key', columns='DEPARTMENT', values='SALES_VALUE').fillna(0)

# 상위 10개 카테고리
top_categories = normal_with_product.groupby('DEPARTMENT')['SALES_VALUE'].sum().sort_values(ascending=False).head(10).index
category_features = category_pivot[top_categories].copy()

# 비율 변환
total_spending_by_customer = category_features.sum(axis=1)
for col in category_features.columns:
    category_features[f'{col}_ratio'] = category_features[col] / total_spending_by_customer

print(f"상위 {len(top_categories)}개 카테고리 특성 생성")

# %%
# 7. 인구통계 특성 처리

print("=== 인구통계 특성 처리 ===")

demo_features = hh_demographic.copy()

# 순서형 변환
age_order = ['19-24', '25-34', '35-44', '45-54', '55-64', '65+']
demo_features['age_numeric'] = demo_features['AGE_DESC'].map({age: i for i, age in enumerate(age_order)})

income_order = ['Under 15K', '15-24K', '25-34K', '35-49K', '50-74K', '75-99K', '100-124K', '125-149K', '150-174K', '175-199K', '200-249K', '250K+']
demo_features['income_numeric'] = demo_features['INCOME_DESC'].map({inc: i for i, inc in enumerate(income_order)})

demo_features['household_size_numeric'] = demo_features['HOUSEHOLD_SIZE_DESC'].map({'1': 1, '2': 2, '3': 3, '4': 4, '5+': 5})
demo_features['kids_numeric'] = demo_features['KID_CATEGORY_DESC'].map({'None/Unknown': 0, '1': 1, '2': 2, '3+': 3})

# 더미 변수
demo_dummies = pd.get_dummies(demo_features[['household_key', 'MARITAL_STATUS_CODE', 'HOMEOWNER_DESC']], 
                              columns=['MARITAL_STATUS_CODE', 'HOMEOWNER_DESC'])

demo_final = demo_features[['household_key', 'age_numeric', 'income_numeric', 'household_size_numeric', 'kids_numeric']].merge(
    demo_dummies, on='household_key'
)

print(f"인구통계 특성 처리 완료: {len(demo_final)}명")

# %%
# 8. 캠페인 참여 특성

print("=== 캠페인 참여 특성 생성 ===")

campaign_participation = campaign_table.groupby('household_key').agg({
    'CAMPAIGN': 'nunique',
    'DESCRIPTION': 'count'
}).rename(columns={'CAMPAIGN': 'unique_campaigns', 'DESCRIPTION': 'total_campaign_participation'})

print(f"캠페인 참여 고객: {len(campaign_participation)}명")

# %%
# 9. 최종 특성 데이터셋 통합 (수정 버전)

print("=== 최종 특성 데이터셋 통합 ===")

final_features = customer_features.copy()

# 각 특성 그룹 결합
feature_groups = [
    category_features,
    discount_features if len(discount_features) > 0 else pd.DataFrame(),
    normal_discount_usage if len(normal_discount_usage) > 0 else pd.DataFrame(),
    free_product_usage if len(free_product_usage) > 0 else pd.DataFrame(),
    campaign_participation
]

for features in feature_groups:
    if len(features) > 0:
        final_features = final_features.join(features, how='left')

# 인구통계 결합
final_features = final_features.merge(demo_final, left_index=True, right_on='household_key', how='left')
final_features.set_index('household_key', inplace=True)

# 결측값 처리 (범주형과 수치형 분리)
print("결측값 처리 중...")

# 수치형 컬럼과 범주형 컬럼 분리
numeric_columns = final_features.select_dtypes(include=[np.number]).columns
categorical_columns = final_features.select_dtypes(exclude=[np.number]).columns

print(f"수치형 컬럼: {len(numeric_columns)}개")
print(f"범주형 컬럼: {len(categorical_columns)}개")

# 수치형 컬럼만 0으로 채우기
if len(numeric_columns) > 0:
    final_features[numeric_columns] = final_features[numeric_columns].fillna(0)

# 범주형 컬럼 처리
for col in categorical_columns:
    if final_features[col].isnull().any():
        print(f"범주형 컬럼 '{col}' 결측값 처리")
        # 범주형의 경우 가장 빈도가 높은 값으로 채우거나 'Unknown'으로 채우기
        mode_value = final_features[col].mode()
        if len(mode_value) > 0:
            final_features[col] = final_features[col].fillna(mode_value[0])
        else:
            # 모든 값이 결측인 경우 첫 번째 카테고리로 채우기
            if hasattr(final_features[col], 'cat') and len(final_features[col].cat.categories) > 0:
                final_features[col] = final_features[col].fillna(final_features[col].cat.categories[0])

print(f"\n최종 특성 데이터셋:")
print(f"  고객 수: {len(final_features):,}명")
print(f"  특성 수: {len(final_features.columns)}개")
print(f"  결측값: {final_features.isnull().sum().sum()}개")

# %%
# 10. 데이터 타입 정리

print("=== 데이터 타입 정리 ===")

# 범주형 데이터를 수치형으로 변환 (나이브 베이즈용)
for col in categorical_columns:
    if col in final_features.columns:
        if hasattr(final_features[col], 'cat'):
            # Categorical 데이터를 수치형 코드로 변환
            final_features[col] = final_features[col].cat.codes
        else:
            # 일반 object 타입을 LabelEncoder로 변환
            from sklearn.preprocessing import LabelEncoder
            le = LabelEncoder()
            final_features[col] = le.fit_transform(final_features[col].astype(str))

print("범주형 데이터를 수치형으로 변환 완료")
print(f"최종 결측값: {final_features.isnull().sum().sum()}개")

피처 엔지니어링 시작! (원본 데이터 + 할인 레코드 활용)
transaction_data: 2,595,732건 (할인 레코드 포함)
고유 가구 수: 2500
분석 기간: 1 ~ 711일
coupon 중복 제거: 124548 → 119384
=== 거래 데이터 분리 ===
정상 구매 거래: 2,581,266건
할인/프로모션 레코드: 14,466건

아웃라이어 제거 기준:
  거래금액 > $29.01
  수량 > 10034
정상 거래 정제: 2,581,266 → 2,564,222건
=== 정상 구매 기반 RFM 특성 생성 ===
RFM 기반 고객 특성: 2500명
=== 할인/프로모션 반응 특성 생성 ===
할인 사용 고객: 1991명
정상 거래 중 할인 사용 고객: 2500명
무료 상품 이용 고객: 1476명
=== 타겟 변수 생성 ===
상위 20% 기준:
  총 구매액: $4726.90
  방문 횟수: 152.0회

타겟 변수 분포:
고가치 (구매액): 20.0%
고가치 (방문횟수): 20.1%
고가치 (복합): 27.2%
고가치 (RFM): 30.3%
=== 카테고리 선호도 특성 생성 ===
상위 10개 카테고리 특성 생성
=== 인구통계 특성 처리 ===
인구통계 특성 처리 완료: 801명
=== 캠페인 참여 특성 생성 ===
캠페인 참여 고객: 1584명
=== 최종 특성 데이터셋 통합 ===
결측값 처리 중...
수치형 컬럼: 63개
범주형 컬럼: 11개
범주형 컬럼 'MARITAL_STATUS_CODE_A' 결측값 처리
범주형 컬럼 'MARITAL_STATUS_CODE_B' 결측값 처리
범주형 컬럼 'MARITAL_STATUS_CODE_U' 결측값 처리
범주형 컬럼 'HOMEOWNER_DESC_Homeowner' 결측값 처리
범주형 컬럼 'HOMEOWNER_DESC_Probable Owner' 결측값 처리
범주형 컬럼 'HOMEOWNER_DESC_Probable Renter' 결측값 처리
범주형 컬럼 'HOMEOWNER_DESC_Renter' 결측

In [3]:
# %%
# 11. 특성 데이터 저장 (다시 실행)

features_path = '../data/features/'

print("=== 특성 데이터 저장 ===")

# 나이브 베이즈용 데이터 준비
feature_columns = [col for col in final_features.columns if not col.startswith('high_value')]
X = final_features[feature_columns]

# 타겟 변수들
targets = final_features[['high_value_monetary', 'high_value_frequency', 'high_value_combined', 'high_value_rfm']]

print(f"특성 데이터: {X.shape}")
print(f"타겟 데이터: {targets.shape}")

# CSV 파일로 저장
X.to_csv(f'{features_path}features_X_v2.csv')
targets.to_csv(f'{features_path}targets_y_v2.csv')

# 전체 특성 데이터도 저장
final_features.to_csv(f'{features_path}customer_features_v2.csv')

print(f"\n저장 완료:")
print(f"  features_X_v2.csv: {X.shape}")
print(f"  targets_y_v2.csv: {targets.shape}")
print(f"  customer_features_v2.csv: {final_features.shape}")

# 파일 존재 확인
import os
files_to_check = ['features_X_v2.csv', 'targets_y_v2.csv', 'customer_features_v2.csv']
for file in files_to_check:
    if os.path.exists(f'{features_path}{file}'):
        print(f"✅ {file} 저장됨")
    else:
        print(f"❌ {file} 저장 실패")

=== 특성 데이터 저장 ===
특성 데이터: (2500, 70)
타겟 데이터: (2500, 4)

저장 완료:
  features_X_v2.csv: (2500, 70)
  targets_y_v2.csv: (2500, 4)
  customer_features_v2.csv: (2500, 74)
✅ features_X_v2.csv 저장됨
✅ targets_y_v2.csv 저장됨
✅ customer_features_v2.csv 저장됨
